In [1]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import numpy as np
import torch.optim as optim
from tqdm import tqdm
import torch
import torch.nn as nn
from sklearn.metrics import classification_report

In [2]:
class FloodAreSegmentation(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_list = [f for f in os.listdir(os.path.join(data_dir, 'Image')) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_name = self.image_list[idx]
        image_path = os.path.join(self.data_dir, 'Image', image_name)
        mask_name = os.path.splitext(image_name)[0] + '.png'
        mask_path = os.path.join(self.data_dir, 'Mask', mask_name)

        image = Image.open(image_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask


data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

In [3]:
data_dir = 'flood-area-segmentation'
dataset = FloodAreSegmentation(data_dir, transform=data_transforms)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [4]:
ENCODER='resnet18'
WEIGHTS='imagenet'

class SegmentationModel(nn.Module):
    def __init__(self):
        super(SegmentationModel,self).__init__()

        self.arc=smp.Unet(
            encoder_name=ENCODER,
            encoder_weights=WEIGHTS,
            in_channels=3,
            classes=1,
            activation=None
        )
    def forward(self,images,masks=None):
        logits=self.arc(images)

        if masks!=None:
            loss1=DiceLoss(mode='binary')(logits,masks)
            loss2=nn.BCEWithLogitsLoss()(logits,masks)
            return logits,loss1,loss2
        return logits

In [6]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

segmentation_model = SegmentationModel()
segmentation_model.load_state_dict(torch.load("Inference/segmentation_model.pt"))
segmentation_model.to(device)


SegmentationModel(
  (arc): Unet(
    (encoder): ResNetEncoder(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1):

In [ ]:
# define the loss function
criterion = nn.BCEWithLogitsLoss()

# define the optimizer
optimizer = optim.Adam(segmentation_model.parameters(), lr=0.001)

# define the number of epochs to train for
num_epochs = 100

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    # create a progress bar using tqdm
    pbar = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, data in pbar:
        # get the inputs and labels and move them to the GPU
        inputs, labels = data

        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward pass
        outputs = segmentation_model(inputs)
        loss = criterion(outputs, labels)

        # compute the accuracy
        predicted = outputs > 0.5
        total += labels.numel()
        correct += (predicted == labels).sum().item()

        # backward pass
        loss.backward()
        optimizer.step()

        # update the progress bar and print statistics
        running_loss += loss.item()
        pbar.set_description(f'Epoch: {epoch + 1}, Loss: {running_loss / (i + 1):.3f}, Accuracy: {correct / total:.3f}')


print('Finished Training')